# OSCCAI Simulation Notebook

## Usage Instructions

This notebook simulates output from an Open Source Collective Constitutional AI (OSCCAI) tool.
Please ensure you have the necessary libraries installed as specified in the `requirements.txt` file.
You will need an OpenAI API key to run this notebook. Set it as an environment variable or enter it when prompted.

**Required Setup:**
- Install required libraries using `pip install -r requirements.txt`
- Set your OpenAI API key:
  - Option 1: Set as an environment variable `OPENAI_API_KEY`
  - Option 2: Enter it when prompted in the notebook

## 1. Setup and Imports

In [ ]:
# Uncomment and run this cell to install required packages
# !pip install numpy pandas matplotlib openai

In [ ]:
# Import required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from getpass import getpass
from IPython.display import display, Markdown
from google.colab import files
import pandas as pd
import numpy as np
from scipy.stats import beta, nbinom
import random
from openai import OpenAI

# Set up OpenAI API key
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"),
api_key=getpass("Please enter your OpenAI API key: "))

## 2. Data Upload and Preprocessing

In [ ]:
# Get actual data to base distributions on
print("Please upload your Polis data CSV file.")
uploaded = files.upload()

# Load the uploaded CSV into a DataFrame
filename = list(uploaded.keys())[0]
data = pd.read_csv(filename)

## 3. Define Simulation Functions

In [ ]:
def fit_beta_distribution(data):
    """Fit a beta distribution to the given data."""
    data_cleaned = data.clip(lower=0.001, upper=0.999)
    a, b, _, _ = beta.fit(data_cleaned, floc=0, fscale=1)
    return a, b

def fit_negative_binomial(data):
    """Fit a negative binomial distribution to the given data."""
    mean = data.mean()
    var = data.var()
    n = (mean ** 2) / (var - mean) if var > mean else 10
    p = mean / (mean + n)
    return n, p

def calculate_statements_per_participant(num_participants):
    """Calculate the number of statements per participant based on actual data."""
    n_comments = data['n-comments']
    n, p = fit_negative_binomial(n_comments)
    return [max(np.random.negative_binomial(n, p), 1) for _ in range(num_participants)]

def calculate_votes_per_participant(num_participants):
    """Calculate the number of votes per participant based on actual data."""
    n_votes = data['n-votes']
    n, p = fit_negative_binomial(n_votes)
    return [max(np.random.negative_binomial(n, p), 1) for _ in range(num_participants)]

def calculate_vote_distribution_per_participant(num_participants):
    """Calculate the vote distribution per participant based on actual data."""
    agree_a, agree_b = fit_beta_distribution(data['% agree'])
    disagree_a, disagree_b = fit_beta_distribution(data['% disagree'])
    
    distributions = []
    for _ in range(num_participants):
        agree = np.random.beta(agree_a, agree_b)
        disagree = np.random.beta(disagree_a, disagree_b)
        pass_prob = max(0, 1 - (agree + disagree))
        total = agree + disagree + pass_prob
        distributions.append([round(agree/total, 3), round(disagree/total, 3), round(pass_prob/total, 3)])
    
    return distributions

## 4. User Input Collection

In [ ]:
print("### Community Information Collection ###")
community_name = input("Enter the community name: ").strip()
community_description = input("Enter the community description: ").strip()
community_goals = input("Enter the community goals for the AI model: ").strip()

# Consolidate inputs into a single community description
full_community_description = f"{community_description}\nGoals: {community_goals}"

print("\n### Simulation Parameters Collection ###")
while True:
    try:
        num_subgroups = int(input("Enter the number of subgroups (G): "))
        if num_subgroups <= 0:
            raise ValueError("Number of subgroups must be positive.")
        break
    except ValueError as e:
        print(f"Invalid input: {e}")

while True:
    try:
        num_participants = int(input("Enter the total number of participants (P): "))
        if num_participants <= 0:
            raise ValueError("Number of participants must be positive.")
        break
    except ValueError as e:
        print(f"Invalid input: {e}")

statement_format = input("Enter the statement format (default: 'The best response is one that...'): ").strip()
if not statement_format:
    statement_format = "The best response is one that..."

## 5. Simulation Setup Calculations

In [ ]:
def calculate_participants_per_group(num_subgroups, num_participants):
    """Distribute participants among groups with variability but no extreme disparities."""
    base = num_participants // num_subgroups
    remainder = num_participants % num_subgroups
    participants_per_group = [base] * num_subgroups
    for i in range(remainder):
        participants_per_group[i] += 1
    # Introduce slight variability
    for i in range(len(participants_per_group)):
        variation = np.random.randint(-base//10, base//10+1)
        participants_per_group[i] = max(1, participants_per_group[i] + variation)
    total_participants = sum(participants_per_group)
    # Adjust if total participants changed due to variability
    if total_participants != num_participants:
        difference = num_participants - total_participants
        for i in range(abs(difference)):
            index = i % num_subgroups
            if difference > 0:
                participants_per_group[index] += 1
            else:
                participants_per_group[index] = max(1, participants_per_group[index] - 1)
    return participants_per_group

participants_per_group = calculate_participants_per_group(num_subgroups, num_participants)
print(f"\nParticipants per group: {participants_per_group}")

## 6. LLM Interaction for Group and Participant Generation

In [ ]:
def get_openai_completion(prompt, max_tokens=1500):
    """Sends a prompt to the OpenAI API and returns the completion."""
    try:
        response = client.chat.completions.create(model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant that outputs JSON-formatted data."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0.7)
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error during OpenAI API call: {e}")
        return None

group_generation_prompt = f"""
You are tasked with creating realistic subgroups and participants for a community simulation. Use the following information to generate detailed descriptions:

Community Description: {full_community_description}
Number of Subgroups: {num_subgroups}
Participants per Group: {participants_per_group}

For each subgroup:
1. Provide a 2-sentence description of the subgroup.
2. Generate the specified number of unique, 1-sentence descriptions of individuals who could belong to this subgroup.

Ensure descriptions are diverse and realistic within the context of the community.

Return your response in the following JSON format:
{{"subgroups": [
    {{"description": "Subgroup description",
    "participants": ["Participant 1 description", "Participant 2 description", ...]}},
    ...]
}}
"""

group_generation_response = get_openai_completion(group_generation_prompt)

try:
    group_data = json.loads(group_generation_response)
    subgroups = group_data['subgroups']
    print("\nGenerated Subgroups and Participants:")
    print(json.dumps(group_data, indent=4))
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    subgroups = []

## 7. LLM Interaction for Statement Generation

In [ ]:
statements_per_participant = calculate_statements_per_participant(num_participants)
subgroups_and_participants_json = json.dumps(group_data, indent=4)

statement_generation_prompt = f"""
Generate statements for a community AI model alignment survey. Use the following information:

Community Description: {full_community_description}
Statement Format: "{statement_format}"

For each participant, generate a unique number of statements that align with their subgroup and individual characteristics. The number of statements for each participant is provided in the JSON below. Ensure statements are diverse and relevant to the community's goals.

Subgroups and Participants:
{subgroups_and_participants_json}

Number of statements per participant:
{json.dumps(dict(zip(range(num_participants), statements_per_participant)))}

Return your response in the following JSON format:
{{
    "statements": [
        {{
            "participant_id": "Unique identifier",
            "statements": ["Statement 1", "Statement 2", ...]
        }},
        ...
    ]
}}
"""

statement_generation_response = get_openai_completion(statement_generation_prompt)

try:
    statement_data = json.loads(statement_generation_response)
    participant_statements = statement_data['statements']
    print("\nGenerated Participant Statements:")
    print(json.dumps(statement_data, indent=4))
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    participant_statements = []

## 8. LLM Interaction for Vote Simulation

In [ ]:
# Prepare numbered statements list
statements_list = []
statement_id = 1
participant_ids = []
for participant in participant_statements:
    participant_ids.append(participant['participant_id'])
    for stmt in participant['statements']:
        statements_list.append({'statement_id': statement_id, 'text': stmt})
        statement_id += 1

numbered_statements_list = json.dumps(statements_list, indent=4)

# Prepare participants and assignments data
votes_per_participant = calculate_votes_per_participant(num_participants)
vote_distributions = calculate_vote_distribution_per_participant(num_participants)

participants_and_assignments = []
for i, participant in enumerate(participant_statements):
    participants_and_assignments.append({
        'participant_id': participant['participant_id'],
        'statements_assigned': random.sample([s['statement_id'] for s in statements_list], votes_per_participant[i]),
        'vote_distribution': dict(zip(['agree', 'disagree', 'pass'], vote_distributions[i]))
    })
participants_and_assignments_json = json.dumps(participants_and_assignments, indent=4)

vote_simulation_prompt = f"""
Simulate voting patterns for a community AI model alignment survey. Use the following information:

Community Description: {full_community_description}
Statements:
{numbered_statements_list}

For each participant, determine how they would likely vote on their assigned statements. Use the following voting options:
1 = Agree
-1 = Disagree
0 = Pass

Participants and Voting Assignments:
{participants_and_assignments_json}

Ensure that each participant's voting pattern closely matches their target vote distribution.

Return your response in the following JSON format:
{{
    "votes": [
        {{
            "participant_id": "Unique identifier",
            "votes": {{"statement_id": vote, ...}}
        }},
        ...
    ]
}}
"""

vote_simulation_response = get_openai_completion(vote_simulation_prompt)

try:
    vote_data = json.loads(vote_simulation_response)
    votes = vote_data['votes']
    print("\nSimulated Votes:")
    print(json.dumps(vote_data, indent=4))
except json.JSONDecodeError as e:
    print(f"Error parsing JSON: {e}")
    votes = []

## 9. Vote Matrix Generation

In [ ]:
# Create empty matrix (rows = participants, columns = statements)
participant_ids = [vote['participant_id'] for vote in votes]
statement_ids = [s['statement_id'] for s in statements_list]

vote_matrix = pd.DataFrame(index=participant_ids, columns=statement_ids)

# Populate matrix with extracted votes
for vote_entry in votes:
    participant_id = vote_entry['participant_id']
    participant_votes = vote_entry['votes']
    for stmt_id, vote_value in participant_votes.items():
        vote_matrix.at[participant_id, int(stmt_id)] = vote_value

# Convert data types
vote_matrix = vote_matrix.apply(pd.to_numeric, errors='coerce')

## 10. Results Visualization and Analysis

In [ ]:
# Generate heatmap of Vote Matrix
plt.figure(figsize=(12, 8))
plt.imshow(vote_matrix.fillna(0), aspect='auto', cmap='bwr', interpolation='none')
plt.colorbar(label='Vote (-1: Disagree, 0: Pass, 1: Agree)')
plt.xlabel('Statements')
plt.ylabel('Participants')
plt.title('Vote Matrix Heatmap')
plt.show()

# Calculate and display vote distribution statistics
vote_counts = vote_matrix.stack().value_counts()
print("\n### Vote Distribution Statistics ###")
print(vote_counts)